In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [208]:
data_c16 = pd.read_excel('../data/2016 Data Cleaned.xlsx')
fea = pd.ExcelFile('../data/FoodEnvironmentAtlas.xls')
supp = fea.parse('Supplemental Data - County')
pop_16 = supp['Population_Estimate_2016']

thx_craig = pd.read_csv('../data/data_train_unscaled.csv')

In [230]:
# find index of bedford county in Craig's data
bedford_good_data = thx_craig[(thx_craig['County'] == 'Bedford') & (thx_craig['State'] == 'VA')]
#bedford_good_data

In [231]:
# replace Bedford county data with Craig's preprocessing
bedford = data_c16[(data_c16["County"]=='Bedford') & (data_c16["State"]== 'VA')]
bedford
df = data_c16.drop(['County', 'Unnamed: 27', 'Unnamed: 28', 'Variable Code', 'Variable Name'], axis = 1)
df.drop(2916, axis = 0, inplace = True)
bedford_ind = 2829

# find which columns are not in Craig's dataset
inconsistent_cols = []
for col in df:
    if col not in thx_craig.columns:
        inconsistent_cols.append(col)

consistent_cols = []
for col in df:
    if col not in inconsistent_cols:
        consistent_cols.append(col)

# Repalce Bedford county's value with sum of Bedford County and Bedford Town as these are not percentages but totals
for col in inconsistent_cols:
    df.loc[2829,col] = data_c16.loc[2829, col] + data_c16.loc[bedford_ind, col]

# Replace other columns with Craig's recalculated values
for col in consistent_cols:
    df.loc[2829,col] = thx_craig.loc[766,col]

# find county in Wyoming with nan population
#data_c16[(data_c16["State"] == 'WY')]

In [234]:
data = pd.concat([df, pop_16], axis = 1)
cols = data.columns[1:-1]
states = data["State"].drop_duplicates()
states.drop(2916, axis = 0, inplace = True)

In [235]:
state_avgs = []

#there was no population data for one of the wyoming counties
# found this from the US Census Bureau 
data.loc[3142, 'Population_Estimate_2016'] = 7228

# Correct Bedford county's population
data.loc[bedford_ind, 'Population_Estimate_2016'] = pop_16.loc[2916] + pop_16.loc[bedford_ind]

for state in states:
    rows = data[data["State"] == state]
    state_avg = np.average(rows[cols], weights = rows['Population_Estimate_2016'], axis = 0)
    state_avgs.append(state_avg)
len(state_avgs)

51

In [236]:
state_feats = pd.DataFrame(state_avgs)
state_feats.columns = cols
state_labs = pd.DataFrame(states)
state_labs.set_index(np.arange(0,51), inplace = True)
state_data = pd.concat([state_labs, state_feats], axis = 1)

In [237]:
state_data.to_csv('../obesity-insecurity/state_data.csv')